### Notebook to compute travel demand with Gravitational Model and travel weights using LOGIT model

#### Initial data

In [8]:
# number of travels passed as a matrix with produced travels on lines and attracted ones on columns

travs = [[40, 110, 150],
         [50, 20, 30],
         [110, 30, 10]]

# the matching friction factors, same arrangement

ffs = [[0.753, 1.597, 0.753],
       [0.987, 0.753, 0.765],
       [1.597, 0.765, 0.753]]

# neutral calibration coefficients

k_ij0 = [[1, 1, 1],
         [1, 1, 1],
         [1, 1, 1]]

# auto travels cost

tca = [[0.5, 1, 1.4],
       [1.2, 0.8, 1.2],
       [1.7, 1.5, 0.7]]

# transit travels cost

tct = [[1, 1.5, 2],
       [1.8, 1.2, 1.9],
       [1.7, 1.5, 0.7]]

# auto travels duration

tda = [[3, 12, 7],
       [13, 3, 19],
       [9, 16, 4]]

# transit travels duration

tdt = [[15, 5, 12],
       [15, 6, 26],
       [20, 21, 8]]

# the future friction factors

ffs_f = [[0.753, 0.987, 1.597],
         [0.987, 0.753, 0.765],
         [1.597, 0.765, 0.753]]

# the future produced travels

P_is = [750, 580, 480]

# the future attracted travels

A_js = [722, 786, 302]

In [9]:
# hist travels matrix AET
travs_aet = [[120, 60],
             [80, 40]]

# friction factors matrix AET
ffs_aet = [[0.852, 1.387],
           [0.796, 0.788]]

# calibration coeffs matrix for hist travels AET
k_ij0_aet = [[1, 1],
             [1, 1]]

# auto travels cost AET

tca_aet = [[0.5, 0.5],
           [0.8, 2.1]]

# transit travels cost AET

tct_aet = [[1, 1.5],
           [1.2, 1.6]]

# auto travels duration AET

tda_aet = [[3, 9],
           [11, 3]]

# transit travels duration AET

tdt_aet = [[15, 6],
           [16, 6]]

#### Class to compute the travels using Gravitational Model

In [10]:
class GravitMod:
    def __init__(self, travs, ffs, k_ijs, P_is, A_js):
        self.travs = travs
        self.ffs = ffs
        self.k_ijs = k_ijs
        self.P_is = P_is
        self.A_js = A_js

    
    def transp_mat(mat):
        """
        Method to transpose a squared matrix.
        Takes as input the matrix, returns the transpose, same shape.
        """
        # check for same number of rows and columns (square matrix)
        if(len(mat) != len(mat[0])):
            print("The matrix is not squared. Please provide a squared matrix.")
            exit()

        # transpose the matrix
        transp = list(zip(*mat))

        return [list(sublist) for sublist in transp]

    
    def sum_mat(mat):
        """
        Method to compute the sum of rows of a squared matrix.
        Takes as input the matrix, returns the sum of rows.
        """

        if(len(mat) != len(mat[0])):
            print("The matrix is not squared. Please provide a squared matrix.")
            exit()

        sum_mat = []    # empty list to store the sums

        for row in mat:
            sum_mat.append(sum(row))

        return sum_mat
    
    
    def formula(s_Pi, s_Aj, ffs, k_ijs, travs):
        """
        Method to apply the Gravitational Model formula.
        Takes as inputs the sum of produced, respectively attracted travels,
        the matrix of friction factors, the matrix of calibration coefficients,
        and the initial travels matrix.
        Returns the squared matrix of computed travels.
        """
        
        gvals_init = []    # to store computed values
        for i in range(len(travs)):
            pdsum = 0
            for j1, j2 in zip(s_Aj, ffs[i]):
                pdsum = pdsum + j1 * j2
            for k1 in range(len(ffs[i])):
                gvals_init.append((s_Pi[i] * ffs[i][k1] * s_Aj[k1] * k_ijs[i][k1] / pdsum))

        return gvals_init
        
    
    def rnd(mat, trail=0):
        """
        Method to round the travels raw computed with Gravitational Model.
        Takes as input the single row matrix, returns rounded values, same shape.
        """
        mat_r = []
        for item in mat:
            mat_r.append(round(item, trail))

        return mat_r
    
    def mat_grp(mat, tlen):
        """
        Method to group the rounded single row matrix into a squared one.
        Takes as input the single row matrix of rounded travels and
        the dimension of the squared one.
        Returns the squared matrix of travels obtained with
        Gravitational Model.
        """
        return [mat[i:i + tlen] for i in range(0, len(mat), tlen)]
    
    
    def gravmod_init(travs, ffs, k_ijs):
        """
        Method to compute gravitational model values in order to determine the
        calibration factors.
        Takes as input the travels, friction factors and calibration
        coefficients matrices.
        Returns a matrix with the computed travels.
        """
    
        # check if the matrices have the same shape
        if(len(travs) != len(ffs) or (len(travs) != len(k_ijs))):
            print("The matrices doesn't match. Please fix it.")
            exit()
    
        # transpose the travels and friction factors matrices
        travs_t = GravitMod.transp_mat(travs)
        ffs_t = GravitMod.transp_mat(ffs)
        #print(travs_t)
    
        # get attracted travels sums
        s_Aj = GravitMod.sum_mat(travs_t)

        # get produced travels sums
        s_Pi = GravitMod.sum_mat(travs)
    
        # compute travels with gravitational model
        gvals_init = GravitMod.formula(s_Pi, s_Aj, ffs, k_ijs, travs)
        print(gvals_init)

        # round the computed single row matrix
        gvals_init_r = GravitMod.rnd(gvals_init)
    
        # group flatten list 'gvals_init_r' as a matrix
        gvals_init_m = GravitMod.mat_grp(gvals_init_r, len(travs))
        
        return gvals_init_m

    
    def gravmod_fin(ffs, k_ijs, P_is, A_js, travs):
        """
        Method to compute future travels using gravitational model.
        Takes as inputs the future friction factors matrix, the previously
        computed calibration coefficients matrix, the matrix of produced
        travels, the matrix of attracted travels and historical travels
        matrix (for shape/number of total travels).
        Returns a matrix with future travels determined with gravitational
        model.
        """
        
        # check if the matrices have the same shape
        if(len(k_ijs) != len(ffs) or (len(k_ijs) != len(P_is)) or \
           (len(k_ijs) != len(A_js))):
            print("The matrices doesn't match. Please fix it.")
            exit()
    
        # compute travels with gravitational model
        gvals_fin = GravitMod.formula(P_is, A_js, ffs, k_ijs, travs)
    
        # round the number of travels
        gvals_fin_r = GravitMod.rnd(gvals_fin)

        # group flatten list 'gvals_fin_r' as a matrix
        gvals_fin_m = GravitMod.mat_grp(gvals_fin_r, len(travs))

        return gvals_fin_m

In [11]:
# compute and print travels using Gravitational Model on historical
# data for later calibration coefficients determination
gvalsr = GravitMod.gravmod_init(travs, ffs, k_ij0)
print("Returned from gravmod_init method, ", gvalsr, '\n')

[82.26661082685409, 139.5801088876345, 78.15328028551139, 42.613820348423026, 26.00867819441746, 31.37750145715951, 81.9156393728521, 31.391591293791787, 36.69276933335614]
Returned from gravmod_init method,  [[82.0, 140.0, 78.0], [43.0, 26.0, 31.0], [82.0, 31.0, 37.0]] 



In [12]:
# compute and print travels using Gravitational Model on historical
# data for later calibration coefficients determination (AET)
#print(travs_aet)
#print(ffs_aet)
#print(k_ij0_aet)
gvalsr_aet = GravitMod.gravmod_init(travs_aet, ffs_aet, k_ij0_aet)
print("Returned from gravmod_init method (AET), ", gvalsr_aet, '\n')

[99.23002264639274, 80.76997735360725, 80.26890756302522, 39.73109243697479]
Returned from gravmod_init method (AET),  [[99.0, 81.0], [80.0, 40.0]] 



#### Class to adjust the travels computed with Gravitational Model

In [13]:
class IterAdj:
    def __init__(self, travs, travsc, tlr):
        self.travs = travs
        self.travsc = travsc
        self.tlr = tlr

    def comp(s_ih, s_ic, tlr):
        """
        Method to compare two values, within tolerance.
        Takes as inputs the lists of to be compared values
        and the precision/tolerance.
        Returns True or False.
        """
            
        # set a flag
        flag = True

        for ih, ic in zip(s_ih, s_ic):
            if(abs(ih - ic) / ih >= tlr): 
                flag = False
                break

        return flag
    
    def iter_adj_in(travs, travsc, tlr=0.01):
        """
        Method to iteratively adjust travels computed with gravitational model.
        Takes as input the observed (historical) travels,the computed
        ones, in the form of matrices and the precision (tolerance) of
        adjustment.
        Returns a matrix with adjusted travels.
        """
    
        # check if the matrices have the same shape
        if(len(travs) != len(travsc)):
            print("The matrices doesn't match. Please fix it.")
            exit()
        
        # get produced travels sums on observed travels
        s_Pih = GravitMod.sum_mat(travs)
        
        # transpose the observed travels matrix
        Ajh = GravitMod.transp_mat(travs)
        
        # get attracted travels sums on observed travels
        s_Ajh = GravitMod.sum_mat(Ajh)

        cmp_flg = False  # comparison flag to govern the following cycle
        i = 0   # produced passes counter
        j = 0   # attracted passes counter

        while(cmp_flg == False):
                       
            # get produced travels sums on computed travels
            s_Pic = GravitMod.sum_mat(travsc)
            
            if (IterAdj.comp(s_Pih, s_Pic, tlr) == False):
                ccsi = []   # list to store produced travels coefficients
                for ph, pc in zip(s_Pih, s_Pic):
                    ccsi.append(round(ph/pc, 3))

                for x in range(len(travsc)):
                    travsc[x] = [ccsi[x]*item for item in travsc[x]]
            
                i += 1

            # transpose de matrix of computed travels
            Ajc = GravitMod.transp_mat(travsc)

            # get attracted travels sums on computed travels
            s_Ajc = GravitMod.sum_mat(Ajc)
            
            if (IterAdj.comp(s_Ajh, s_Ajc, tlr) == False):
                ccsj = []   # list to store attracted travels coefficients
                for ah, ac in zip(s_Ajh, s_Ajc):
                    ccsj.append(round(ah/ac, 3))

                for x in range(len(Ajc)):
                    Ajc[x] = [ccsj[x]*item for item in Ajc[x]]
            
                j += 1

            travsc = GravitMod.transp_mat(Ajc)    # transpose the transposed

            # reduced trailing digits
            travsc_r = [[], [], []]
            for row in range(len(travsc)):
                travsc_r[row] = GravitMod.rnd(travsc[row], trail=2)

            print("Travels matrix (trailing 2 digits) after i = ", i, "and j = ", j, "is ", travsc_r)
                
            # get attracted travels sums on new computed travels matrix
            # transpose de matrix of computed travels
            Ajc = GravitMod.transp_mat(travsc)

            # get attracted travels sums on computed travels
            s_Ajc = GravitMod.sum_mat(Ajc)

            # update the produced sums
            # get produced travels sums on computed travels
            s_Pic = GravitMod.sum_mat(travsc)
        
            cmp_flg = IterAdj.comp(s_Ajh, s_Ajc, tlr)
                        
            cmp_flg = IterAdj.comp(s_Pih, s_Pic, tlr)
            
            # round the computed travels
            for row in range(len(travsc)):
                travsc[row] = GravitMod.rnd(travsc[row])
        
        return travsc

    def iter_adj_fin(travsc, P_is, A_js, tlr=0.01):
        """
        Method to iteratively adjust future travels computed with 
        gravitational model.
        Takes as input the computed ones, in the form of matrices,
        the future produced and attracted travels,
        and the precision (tolerance) of adjustment.
        Returns a matrix with adjusted travels.
        """
    
        # get produced travels sums on observed travels
        s_Pih = P_is
        
        # get attracted travels sums on observed travels
        s_Ajh = A_js

        cmp_flg = False  # comparison flag to govern the following cycle
        i = 0   # produced passes counter
        j = 0   # attracted passes counter

        while(cmp_flg == False):
                       
            # get produced travels sums on computed travels
            s_Pic = GravitMod.sum_mat(travsc)
            
            if (IterAdj.comp(s_Pih, s_Pic, tlr) == False):
                ccsi = []   # list to store produced travels coefficients
                for ph, pc in zip(s_Pih, s_Pic):
                    ccsi.append(round(ph/pc, 3))

                for x in range(len(travsc)):
                    travsc[x] = [ccsi[x]*item for item in travsc[x]]
            
                i += 1

            # transpose de matrix of computed travels
            Ajc = GravitMod.transp_mat(travsc)

            # get attracted travels sums on computed travels
            s_Ajc = GravitMod.sum_mat(Ajc)

            if (IterAdj.comp(s_Ajh, s_Ajc, tlr) == False):
                ccsj = []   # list to store attracted travels coefficients
                for ah, ac in zip(s_Ajh, s_Ajc):
                    ccsj.append(round(ah/ac, 3))

                for x in range(len(Ajc)):
                    Ajc[x] = [ccsj[x]*item for item in Ajc[x]]
            
                j += 1

            travsc = GravitMod.transp_mat(Ajc)    # transpose the transposed

            # reduced trailing digits
            travsc_r = [[], [], []]
            for row in range(len(travsc)):
                travsc_r[row] = GravitMod.rnd(travsc[row], trail=2)

            print("Future travels matrix (trailing 2 digits) after i = ", i, "and j = ", j, "is ", travsc_r)
                
            # get attracted travels sums on new computed travels matrix
            # transpose de matrix of computed travels
            Ajc = GravitMod.transp_mat(travsc)

            # get attracted travels sums on computed travels
            s_Ajc = GravitMod.sum_mat(Ajc)

            # update the produced sums
            # get produced travels sums on computed travels
            s_Pic = GravitMod.sum_mat(travsc)
        
            cmp_flg = IterAdj.comp(s_Ajh, s_Ajc, tlr)
                        
            cmp_flg = IterAdj.comp(s_Pih, s_Pic, tlr)
            
            # round the computed travels
            for row in range(len(travsc)):
                travsc[row] = GravitMod.rnd(travsc[row])
        
        return travsc

In [14]:
travsc_adj = IterAdj.iter_adj_in(travs, gvalsr)
print("Historical computed and adjusted, ", travsc_adj)
print("Historical travels matrix, ", travs)

Travels matrix (trailing 2 digits) after i =  0 and j =  1 is  [[79.21, 113.68, 101.48], [41.54, 21.11, 40.33], [79.21, 25.17, 48.14]]
Travels matrix (trailing 2 digits) after i =  1 and j =  1 is  [[80.58, 116.28, 103.02], [40.78, 20.39, 38.84], [77.97, 24.68, 47.38]]
Historical computed and adjusted,  [[81.0, 116.0, 103.0], [41.0, 20.0, 39.0], [78.0, 25.0, 47.0]]
Historical travels matrix,  [[40, 110, 150], [50, 20, 30], [110, 30, 10]]


In [15]:
travsc_adj_aet = IterAdj.iter_adj_in(travs_aet, gvalsr_aet)
print("Historical computed and adjusted, ", travsc_adj_aet)
print("Historical travels matrix, ", travs_aet)

Travels matrix (trailing 2 digits) after i =  0 and j =  1 is  [[110.58, 66.91], [89.36, 33.04], []]
Travels matrix (trailing 2 digits) after i =  1 and j =  1 is  [[112.22, 67.74], [87.58, 32.47], []]
Historical computed and adjusted,  [[112.0, 68.0], [88.0, 32.0]]
Historical travels matrix,  [[120, 60], [80, 40]]


#### Function to compute the calibration coefficients

In [16]:
def ccoeffs(travs, travsc):
    """
    Function to compute calibration coefficients for Gravitational Model.
    Takes as inputs computed-adjusted travels, and observed ones.
    Returns a squared matrix with calibration coefficients.
    """
    ccoeffs = []
    for row_h, row_c in zip(travs, travsc):
        for t_h, t_c in zip(row_h, row_c):
            ccoeffs.append(round(t_h / t_c, 2))
    
    ccoeffs_m = GravitMod.mat_grp(ccoeffs, len(travs))

    return ccoeffs_m

In [17]:
# get the calibration coeffs
cal_coeffs = ccoeffs(travs, travsc_adj)
print("Calibration coefficients matrix, ", cal_coeffs)

Calibration coefficients matrix,  [[0.49, 0.95, 1.46], [1.22, 1.0, 0.77], [1.41, 1.2, 0.21]]


In [18]:
# get the calibration coeffs AET
cal_coeffs_aet = ccoeffs(travs_aet, travsc_adj_aet)
print("Calibration coefficients matrix AET, ", cal_coeffs_aet)

Calibration coefficients matrix AET,  [[1.07, 0.88], [0.91, 1.25]]


In [19]:
# call the GravitMod future travels method 
# with calibration coefficients and future produced, respectively attracted travels
gravmod_fin = GravitMod.gravmod_fin(ffs_f, cal_coeffs, P_is, A_js, travs)
print("Future travels computed with Gravitational Model and initial data, ", gravmod_fin, '\n')

Future travels computed with Gravitational Model and initial data,  [[111.0, 307.0, 293.0], [328.0, 224.0, 67.0], [394.0, 175.0, 12.0]] 



In [20]:
print("Sums of produced on future travels, ", GravitMod.sum_mat(gravmod_fin),'\n')

Sums of produced on future travels,  [711.0, 619.0, 581.0] 



In [21]:
print("Sums of attracted on future travels, ", GravitMod.sum_mat(GravitMod.transp_mat(gravmod_fin)),'\n')

Sums of attracted on future travels,  [833.0, 706.0, 372.0] 



In [22]:
gravmod_fin_adj = IterAdj.iter_adj_fin(gravmod_fin, P_is, A_js)
print("Future travels adjusted, ", gravmod_fin_adj,'\n')

Future travels matrix (trailing 2 digits) after i =  1 and j =  1 is  [[112.77, 375.38, 244.51], [295.96, 243.26, 49.66], [313.4, 167.53, 7.84]]
Future travels matrix (trailing 2 digits) after i =  2 and j =  2 is  [[116.75, 382.86, 245.87], [294.48, 238.88, 48.31], [310.44, 164.65, 7.71]]
Future travels adjusted,  [[117.0, 383.0, 246.0], [294.0, 239.0, 48.0], [310.0, 165.0, 8.0]] 



In [23]:
print("Sums of produced on future travels, ", GravitMod.sum_mat(gravmod_fin_adj),'\n')
print("Targeted sums of produced, ", P_is,'\n')
print("*****", '\n')
print("Sums of attracted on future travels, ", GravitMod.sum_mat(GravitMod.transp_mat(gravmod_fin_adj)),'\n')
print("Targeted sums of attracted, ", A_js,'\n')

Sums of produced on future travels,  [746.0, 581.0, 483.0] 

Targeted sums of produced,  [750, 580, 480] 

***** 

Sums of attracted on future travels,  [721.0, 787.0, 302.0] 

Targeted sums of attracted,  [722, 786, 302] 



#### Function to determine modal option

In [24]:
def modopt(tca, tct, tda, tdt):
    """
    Function to compute modal option, i.e. auto and transit.
    Takes as inputs the matrices of travels cost, auto and transit, and
    duration, respectively.
    Returns the utilities of auto and transit travels for each zone to zone
    combination.
    """
    
    # compute utilities for auto and transit modes
    u_a = []    # store auto utility results
    u_t = []    # store transit utility results

    for i in range(len(tca)):
        for ca, da in zip(tca[i], tda[i]):
            u_a.append(2.5 - 0.5 * ca - 0.01 * da)
        for ct, dt in zip(tct[i], tdt[i]):
            u_t.append(-0.4 * ct - 0.012 * dt)

    return (GravitMod.rnd(u_a, 4), GravitMod.rnd(u_t, 4))

In [25]:
u_a, u_t = modopt(tca, tct, tda, tdt)
u_a_mat = GravitMod.mat_grp(u_a, len(tca))
u_t_mat = GravitMod.mat_grp(u_t, len(tct))
print("Matrix utilities for auto and transit sub-modes, u_a_mat = ", u_a_mat, "and u_t_mat = ", u_t_mat, '\n')

Matrix utilities for auto and transit sub-modes, u_a_mat =  [[2.22, 1.88, 1.73], [1.77, 2.07, 1.71], [1.56, 1.59, 2.11]] and u_t_mat =  [[-0.58, -0.66, -0.944], [-0.9, -0.552, -1.072], [-0.92, -0.852, -0.376]] 



In [26]:
u_a_aet, u_t_aet = modopt(tca_aet, tct_aet, tda_aet, tdt_aet)
u_a_aet_mat = GravitMod.mat_grp(u_a_aet, len(tca_aet))
u_t_aet_mat = GravitMod.mat_grp(u_t_aet, len(tct_aet))
print("Matrix utilities for auto and transit sub-modes, AET, u_a_aet_mat = ", u_a_aet_mat, "and u_t_aet_mat = ", u_t_aet_mat, '\n')

Matrix utilities for auto and transit sub-modes, AET, u_a_aet_mat =  [[2.22, 2.16], [1.99, 1.42]] and u_t_aet_mat =  [[-0.58, -0.672], [-0.672, -0.712]] 



#### Function to compute auto and transit weights, from to each zone

In [27]:
def logit(u_a, u_t):
    """
    Function to compute travels weights for each zone.
    Takes as inputs the utilities lists, auto and transit.
    Returns auto and transit weights for each zone to zone combination.
    """
    # import to get Euler number
    from math import e
    
    w_a = []    # store auto weights
    w_t = []    # store transit weights
    for u_a, u_t in zip(u_a, u_t):
        w_i = e**u_a / (e**u_a + e**u_t)
        w_a.append(round(w_i, 2))
        w_t.append(round(1-w_i, 2))

    return (w_a, w_t)

In [28]:
w_a, w_t = logit(u_a, u_t)
w_a_mat = GravitMod.mat_grp(w_a, len(travs))
w_t_mat = GravitMod.mat_grp(w_t, len(travs))
print("Weights for each zone, ", '\n', "AUTO ", w_a_mat, '\n',"TRAN ", w_t_mat)

Weights for each zone,  
 AUTO  [[0.94, 0.93, 0.94], [0.94, 0.93, 0.94], [0.92, 0.92, 0.92]] 
 TRAN  [[0.06, 0.07, 0.06], [0.06, 0.07, 0.06], [0.08, 0.08, 0.08]]


In [31]:
w_a_aet, w_t_aet = logit(u_a_aet, u_t_aet)
# print(w_a_aet)
w_a_aet_mat = GravitMod.mat_grp(w_a_aet, len(travs_aet))
w_t_aet_mat = GravitMod.mat_grp(w_t_aet, len(travs_aet))
print("Weights for each zone, AET, ", '\n', "AUTO ", w_a_aet_mat, '\n',"TRAN ", w_t_aet_mat)

Weights for each zone, AET,  
 AUTO  [[0.94, 0.94], [0.93, 0.89]] 
 TRAN  [[0.06, 0.06], [0.07, 0.11]]


In [30]:
# allocate AET matrix travels on sub-modes
travs_auto_aet_raw = []
for row_t, row_a in zip(travs_aet, w_a_aet_mat):
    for travel, aweight in zip(row_t, row_a):
        travs_auto_aet_raw.append(travel * aweight)

travs_auto_aet = GravitMod.rnd(travs_auto_aet_raw)
travs_auto_aet_mat = GravitMod.mat_grp(travs_auto_aet, len(travs_aet))

travs_trans_aet_raw = []
for row_t, row_ta in zip(travs_aet, travs_auto_aet_mat):
    for t, ta in zip(row_t, row_ta):
        travs_trans_aet_raw.append(t - ta)

travs_trans_aet_mat = GravitMod.mat_grp(travs_trans_aet_raw, len(travs_aet))

print("Auto travels matrix, AET, ", travs_auto_aet_mat,'\n')
print("Transit travels matrix, AET, ", travs_trans_aet_mat,'\n')

Auto travels matrix, AET,  [[113.0, 56.0], [74.0, 36.0]] 

Transit travels matrix, AET,  [[7.0, 4.0], [6.0, 4.0]] 

